In [ ]:
import pandas as pd

df_preview = pd.read_csv("en.openfoodfacts.org.products.csv.gz", sep='\t', nrows=50)
pd.set_option("display.max_columns", None)  
df_preview.head()


,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,last_modified_by,last_updated_t,last_updated_datetime,product_name,abbreviated_product_name,generic_name,quantity,packaging,packaging_tags,packaging_en,packaging_text,brands,brands_tags,brands_en,categories,categories_tags,categories_en,origins,origins_tags,origins_en,manufacturing_places,manufacturing_places_tags,labels,labels_tags,labels_en,emb_codes,emb_codes_tags,first_packaging_code_geo,cities,cities_tags,purchase_places,stores,countries,countries_tags,countries_en,ingredients_text,ingredients_tags,ingredients_analysis_tags,allergens,allergens_en,traces,traces_tags,traces_en,serving_size,serving_quantity,no_nutrition_data,additives_n,additives,additives_tags,additives_en,nutriscore_score,nutriscore_grade,nova_group,pnns_groups_1,pnns_groups_2,food_groups,food_groups_tags,food_groups_en,states,states_tags,states_en,brand_owner,environmental_score_score,environmental_score_grade,nutrient_levels_tags,product_quantity,owner,data_quality_errors_tags,unique_scans_n,popularity_tags,completeness,last_image_t,last_image_datetime,main_category,main_category_en,image_url,image_small_url,image_ingredients_url,image_ingredients_small_url,image_nutrition_url,image_nutrition_small_url,energy-kj_100g,energy-kcal_100g,energy_100g,energy-from-fat_100g,fat_100g,saturated-fat_100g,butyric-acid_100g,caproic-acid_100g,caprylic-acid_100g,capric-acid_100g,lauric-acid_100g,myristic-acid_100g,palmitic-acid_100g,stearic-acid_100g,arachidic-acid_100g,behenic-acid_100g,lignoceric-acid_100g,cerotic-acid_100g,montanic-acid_100g,melissic-acid_100g,unsaturated-fat_100g,monounsaturated-fat_100g,omega-9-fat_100g,polyunsaturated-fat_100g,omega-3-fat_100g,omega-6-fat_100g,alpha-linolenic-acid_100g,eicosapentaenoic-acid_100g,docosahexaenoic-acid_100g,linoleic-acid_100g,arachidonic-acid_100g,gamma-linolenic-acid_100g,dihomo-gamma-linolenic-acid_100g,oleic-acid_100g,elaidic-acid_100g,gondoic-acid_100g,mead-acid_100g,erucic-acid_100g,nervonic-acid_100g,trans-fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,added-sugars_100g,sucrose_100g,glucose_100g,fructose_100g,lactose_100g,maltose_100g,maltodextrins_100g,starch_100g,polyols_100g,erythritol_100g,fiber_100g,soluble-fiber_100g,insoluble-fiber_100g,proteins_100g,casein_100g,serum-proteins_100g,nucleotides_100g,salt_100g,added-salt_100g,sodium_100g,alcohol_100g,vitamin-a_100g,beta-carotene_100g,vitamin-d_100g,vitamin-e_100g,vitamin-k_100g,vitamin-c_100g,vitamin-b1_100g,vitamin-b2_100g,vitamin-pp_100g,vitamin-b6_100g,vitamin-b9_100g,folates_100g,vitamin-b12_100g,biotin_100g,pantothenic-acid_100g,silica_100g,bicarbonate_100g,potassium_100g,chloride_100g,calcium_100g,phosphorus_100g,iron_100g,magnesium_100g,zinc_100g,copper_100g,manganese_100g,fluoride_100g,selenium_100g,chromium_100g,molybdenum_100g,iodine_100g,caffeine_100g,taurine_100g,ph_100g,fruits-vegetables-nuts_100g,fruits-vegetables-nuts-dried_100g,fruits-vegetables-nuts-estimate_100g,fruits-vegetables-nuts-estimate-from-ingredients_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,carbon-footprint-from-meat-or-fish_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g,choline_100g,phylloquinone_100g,beta-glucan_100g,inositol_100g,carnitine_100g,sulphate_100g,nitrate_100g,acidity_100g
0,54,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1582569031,2020-02-24T18:30:31Z,1733085204,2024-12-01T20:33:24Z,NaN,1740205422,2025-02-22T06:23:42Z,Limonade artisanale a la rose,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en:fr,en:france,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unknown,NaN,unknown,unknown,NaN,NaN,NaN,"en:to-be-completed, en:nutrition-facts-to-be-c...","en:to-be-completed,en:nutrition-facts-to-be-co...","To be completed,Nutrition facts to be complete...",NaN,NaN,unknown,NaN,NaN,NaN,NaN,

In [5]:
import pandas as pd

# =====================
# 配置区（可修改）
# =====================
file_path = "en.openfoodfacts.org.products.csv.gz"  # 你的文件路径
sample_frac = 0.2    # 每类保留的比例
min_samples_per_class = 100  # 排除样本太少的类
chunksize = 10000    # 每次读取的行数
output_file = "food_sampled_by_category.csv"

# =====================
# 所需字段
# =====================
cols = [
    'product_name', 'pnns_groups_1', 'nutriscore_grade',
    'energy-kcal_100g', 'fat_100g', 'saturated-fat_100g',
    'sugars_100g', 'fiber_100g', 'proteins_100g', 'salt_100g'
]

# =====================
# 步骤 1：逐块读取并清洗
# =====================
clean_chunks = []

for chunk in pd.read_csv(file_path, sep='\t', usecols=lambda c: c in cols, chunksize=chunksize, low_memory=False):
    chunk = chunk.dropna(subset=[
        'pnns_groups_1', 'nutriscore_grade',
        'energy-kcal_100g', 'fat_100g', 'saturated-fat_100g',
        'sugars_100g', 'fiber_100g', 'proteins_100g', 'salt_100g'
    ])
    chunk = chunk[chunk['pnns_groups_1'].str.lower() != 'unknown']
    clean_chunks.append(chunk)

df_all = pd.concat(clean_chunks, ignore_index=True)
print(f"清洗后数据总量: {df_all.shape}")

# =====================
# 步骤 2：剔除样本量过小的类
# =====================
valid_classes = df_all['pnns_groups_1'].value_counts()
valid_classes = valid_classes[valid_classes >= min_samples_per_class].index.tolist()
df_all = df_all[df_all['pnns_groups_1'].isin(valid_classes)]

# =====================
# 步骤 3：每个分类采样 20%
# =====================
df_sampled = df_all.groupby('pnns_groups_1', group_keys=False).apply(
    lambda g: g.sample(frac=sample_frac, random_state=42)
).reset_index(drop=True)

print("每类采样完成 ✅")
print(df_sampled['pnns_groups_1'].value_counts())

# =====================
# 步骤 4：保存
# =====================
df_sampled.to_csv(output_file, index=False)
print(f"采样数据已保存为：{output_file}")


清洗后数据总量: (509869, 10)


C:\Users\Administrator\AppData\Local\Temp\ipykernel_10140\3898404145.py:48: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_sampled = df_all.groupby('pnns_groups_1', group_keys=False).apply(


每类采样完成 ✅
pnns_groups_1
Sugary snacks              21942
Cereals and potatoes       17924
Composite foods            10571
Milk and dairy products    10001
Beverages                   9752
Fat and sauces              8903
Fish Meat Eggs              8845
Fruits and vegetables       7838
Salty snacks                5994
Alcoholic beverages          205
Name: count, dtype: int64
采样数据已保存为：food_sampled_by_category.csv


In [ ]:
import pandas as pd
import os

# === 配置文件路径 ===
input_path = "data/raw-data/food_sampled_by_category.csv"
output_path = "data/processed-data/food_sampled_valid_grades.csv"

# === 读取数据 ===
df = pd.read_csv(input_path)

# === 清洗 NutriScore ===
valid_grades = ['a', 'b', 'c', 'd', 'e']
df_cleaned = df[df['nutriscore_grade'].isin(valid_grades)].reset_index(drop=True)

# === 如果没有 processed-data 文件夹就创建 ===
os.makedirs("data/processed-data", exist_ok=True)

# === 保存清洗后的数据 ===
df_cleaned.to_csv(output_path, index=False)

print(f"Cleaning completed, a total of {df_cleaned.shape[0]} records retained, saved to: {output_path}")
